In [100]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy, NMF
from surprise.model_selection import KFold, RepeatedKFold, cross_validate, GridSearchCV

In [36]:
books_df = pd.read_csv("../datasets/clean/filtered_datasets/books_merged.csv")
ratings_df = pd.read_csv("../datasets/clean/filtered_datasets/ratings_filtered_1_to_10_unskewed.csv")

In [58]:
def print_book_titles_by_isbns(isbn_list, books_df):
    # Create a set of unique ISBNs to search for
    isbn_set = set(isbn_list)
    
    # Filter the DataFrame to include only rows with ISBNs in the isbn_set
    matching_books = books_df[books_df['ISBN'].isin(isbn_set)]
    
    # Iterate over matching books and print titles
    for isbn in isbn_list:
        book = matching_books[matching_books['ISBN'] == isbn]
        if not book.empty:
            title = book.iloc[0]['Book-Title']  # Get the book title
            print(f"ISBN: {isbn} - Title: {title}")
        else:
            print(f"ISBN: {isbn} - Title not found")


In [66]:
# Collaborative Filtering
svd = SVD()
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ratings_df[['User-ID', 
                                  'ISBN', 
                                  'Book-Rating']], reader)

In [30]:
trainset = data.build_full_trainset()

In [79]:
svd.fit(trainset)

In [122]:
def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = svd.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

In [123]:
recommendations = get_collaborative_filtering_recommendations(8801,10)
print_book_titles_by_isbns(recommendations,books_df)

ISBN: 0345339738 - Title: The Return of the King (The Lord of the Rings, Part 3)
ISBN: 0439064864 - Title: Harry Potter and the Chamber of Secrets (Book 2)
ISBN: 0836204387 - Title: The Calvin and Hobbes Tenth Anniversary Book
ISBN: 0345339711 - Title: The Two Towers (The Lord of the Rings, Part 2)
ISBN: 0836218051 - Title: The Essential Calvin and Hobbes
ISBN: 0394800168 - Title: Green Eggs and Ham (I Can Read It All by Myself Beginner Books)
ISBN: 0743437810 - Title: The Vanished Man : A Lincoln Rhyme Novel (Lincoln Rhyme Novels (Paperback))
ISBN: 0836221362 - Title: It's A Magical World: A Calvin and Hobbes Collection
ISBN: 0380813815 - Title: Lamb : The Gospel According to Biff, Christ's Childhood Pal
ISBN: 1563893339 - Title: Death : The Time of Your Life (Death)
